## Used Car Price Prediction

Given *data about used cars*, let's try to predict the **price** of a given car.

We will use linear regression and gradient boosting (LightGBM) to make our predictions.

Data source: https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
import lightgbm as lgb

from sklearn.metrics import mean_squared_error

In [24]:
df = pd.read_csv('vehicles.csv', nrows=100000)
df

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,7309358353,https://jacksonville.craigslist.org/ctd/d/jack...,jacksonville,https://jacksonville.craigslist.org,49395,2019.0,chevrolet,silverado trail boss,new,8 cylinders,gas,12700.0,clean,automatic,1GCPYCEF1KZ338219,4wd,full-size,pickup,red,https://images.craigslist.org/00x0x_cg9hmWpOhy...,"*** MINT CONDITION *** BRAND NEW 7"" PRO COMP. ...",NaN,fl,30.207288,-81.738969,2021-04-19T13:50:58-0400
99996,7309358251,https://jacksonville.craigslist.org/ctd/d/jack...,jacksonville,https://jacksonville.craigslist.org,18590,2018.0,kia,sportage lx sport utility 4d,good,NaN,other,20005.0,clean,other,KNDPMCACXJ7395729,NaN,NaN,other,silver,https://images.craigslist.org/00o0o_4QCkqtOcFy...,Carvana is the safer way to buy a car During t...,NaN,fl,30.330000,-81.650000,2021-04-19T13:50:50-0400
99997,7309355294,https://jacksonville.craigslist.org/ctd/d/jack...,jacksonville,https://jacksonville.craigslist.org,49495,2019.0,chevrolet,silverado trail boss,new,8 cylinders,gas,12700.0,clean,automatic,1GCPYCEF1KZ338219,4wd,full-size,pickup,red,https://images.craigslist.org/00x0x_cg9hmWpOhy...,"*** MINT CONDITION *** BRAND NEW 7"" PRO COMP. ...",NaN,fl,30.207288,-81.738969,2021-04-19T13:46:36-0400
99998,7309354677,https://jacksonville.craigslist.org/ctd/d/jack...,jacksonville,https://jacksonville.craigslist.org,24495,2014.0,chevrolet,silverado 1500 lt 4x4,excellent,8 cylinders,gas,100166.0,clean,automatic,1GCVKREC7EZ391819,4wd,full-size,truck,white,https://images.craigslist.org/00f0f_djNCWjHFxp...,*** MINT CONDITION *** CLEAN CARFAX - NO ACCID...,NaN,fl,30.207288,-81.738969,2021-04-19T13:45:40-0400


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            100000 non-null  int64  
 1   url           100000 non-null  object 
 2   region        100000 non-null  object 
 3   region_url    100000 non-null  object 
 4   price         100000 non-null  int64  
 5   year          99572 non-null   float64
 6   manufacturer  95580 non-null   object 
 7   model         98715 non-null   object 
 8   condition     61711 non-null   object 
 9   cylinders     59841 non-null   object 
 10  fuel          99397 non-null   object 
 11  odometer      98479 non-null   float64
 12  title_status  97721 non-null   object 
 13  transmission  99486 non-null   object 
 14  VIN           63962 non-null   object 
 15  drive         69526 non-null   object 
 16  size          27846 non-null   object 
 17  type          80389 non-null   object 
 18  paint

### Preprocessing

In [26]:
df.isna().mean()

id              0.00000
url             0.00000
region          0.00000
region_url      0.00000
price           0.00000
year            0.00428
manufacturer    0.04420
model           0.01285
condition       0.38289
cylinders       0.40159
fuel            0.00603
odometer        0.01521
title_status    0.02279
transmission    0.00514
VIN             0.36038
drive           0.30474
size            0.72154
type            0.19611
paint_color     0.29776
image_url       0.00038
description     0.00039
county          1.00000
state           0.00000
lat             0.00539
long            0.00539
posting_date    0.00038
dtype: float64

In [27]:
null_columns = df.columns[df.isna().mean() > 0.25]
df = df.drop(null_columns, axis=1)

In [28]:
df

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,type,image_url,description,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,7309358353,https://jacksonville.craigslist.org/ctd/d/jack...,jacksonville,https://jacksonville.craigslist.org,49395,2019.0,chevrolet,silverado trail boss,gas,12700.0,clean,automatic,pickup,https://images.craigslist.org/00x0x_cg9hmWpOhy...,"*** MINT CONDITION *** BRAND NEW 7"" PRO COMP. ...",fl,30.207288,-81.738969,2021-04-19T13:50:58-0400
99996,7309358251,https://jacksonville.craigslist.org/ctd/d/jack...,jacksonville,https://jacksonville.craigslist.org,18590,2018.0,kia,sportage lx sport utility 4d,other,20005.0,clean,other,other,https://images.craigslist.org/00o0o_4QCkqtOcFy...,Carvana is the safer way to buy a car During t...,fl,30.330000,-81.650000,2021-04-19T13:50:50-0400
99997,7309355294,https://jacksonville.craigslist.org/ctd/d/jack...,jacksonville,https://jacksonville.craigslist.org,49495,2019.0,chevrolet,silverado trail boss,gas,12700.0,clean,automatic,pickup,https://images.craigslist.org/00x0x_cg9hmWpOhy...,"*** MINT CONDITION *** BRAND NEW 7"" PRO COMP. ...",fl,30.207288,-81.738969,2021-04-19T13:46:36-0400
99998,7309354677,https://jacksonville.craigslist.org/ctd/d/jack...,jacksonville,https://jacksonville.craigslist.org,24495,2014.0,chevrolet,silverado 1500 lt 4x4,gas,100166.0,clean,automatic,truck,https://images.craigslist.org/00f0f_djNCWjHFxp...,*** MINT CONDITION *** CLEAN CARFAX - NO ACCID...,fl,30.207288,-81.738969,2021-04-19T13:45:40-0400


In [29]:
unneeded_columns = ['id', 'url', 'region_url', 'image_url', 'description', 'posting_date']
df = df.drop(unneeded_columns, axis=1)

In [30]:
df

,region,price,year,manufacturer,model,fuel,odometer,title_status,transmission,type,state,lat,long
0,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN
1,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN
2,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN
3,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN
4,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,jacksonville,49395,2019.0,chevrolet,silverado trail boss,gas,12700.0,clean,automatic,pickup,fl,30.207288,-81.738969
99996,jacksonville,18590,2018.0,kia,sportage lx sport utility 4d,other,20005.0,clean,other,other,fl,30.330000,-81.650000
99997,jacksonville,49495,2019.0,chevrolet,silverado trail boss,gas,12700.0,clean,automatic,pickup,fl,30.207288,-81.738969
99998,jacksonville,24495,2014.0,chevrolet,silverado 1500 lt 4x4,gas,100166.0,clean,automatic,truck,fl,30.207288,-81.738969


In [31]:
{column: len(df[column].unique()) for column in df.columns if df.dtypes[column] == 'object'}

{'region': 84,
 'manufacturer': 42,
 'model': 12616,
 'fuel': 6,
 'title_status': 7,
 'transmission': 4,
 'type': 14,
 'state': 18}

In [32]:
df = df.drop('model', axis=1)

In [33]:
def onehot_encode(df, columns, prefixes):
    df = df.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(df[column], prefix=prefix)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    return df

In [34]:
{column: len(df[column].unique()) for column in df.columns if df.dtypes[column] == 'object'}

{'region': 84,
 'manufacturer': 42,
 'fuel': 6,
 'title_status': 7,
 'transmission': 4,
 'type': 14,
 'state': 18}

In [35]:
df = onehot_encode(
    df,
    ['region', 'fuel', 'manufacturer', 'title_status', 'transmission', 'type', 'state'],
    ['reg', 'fuel', 'man', 'title', 'trans', 'type', 'state']
)

In [36]:
df

,price,year,odometer,lat,long,reg_SF bay area,reg_anchorage / mat-su,reg_auburn,reg_bakersfield,reg_bellingham,reg_birmingham,reg_boulder,reg_chico,reg_colorado springs,reg_daytona beach,reg_delaware,reg_denver,reg_dothan,reg_eastern CO,reg_eastern CT,reg_el paso,reg_erie,reg_fairbanks,reg_fayetteville,reg_flagstaff / sedona,reg_florence / muscle shoals,reg_florida keys,reg_fort collins / north CO,reg_fort smith,reg_fresno / madera,reg_ft myers / SW florida,reg_gadsden-anniston,reg_gainesville,reg_gold country,reg_greensboro,reg_hanford-corcoran,reg_hartford,reg_heartland florida,reg_high rockies,reg_hudson valley,reg_humboldt county,reg_huntsville / decatur,reg_imperial county,reg_inland empire,reg_jacksonville,reg_jonesboro,reg_kenai peninsula,reg_la crosse,reg_little rock,reg_los angeles,reg_medford-ashland,reg_mendocino county,reg_merced,reg_mobile,reg_modesto,reg_mohave county,reg_monterey bay,reg_montgomery,reg_new haven,reg_northwest CT,reg_orange county,reg_palm springs,reg_phoenix,reg_prescott,reg_pueblo,reg_redding,reg_reno / tahoe,reg_sacramento,reg_san diego,reg_san luis obispo,reg_santa barbara,reg_santa maria,reg_show low,reg_sierra vista,reg_siskiyou county,reg_skagit / island / SJI,reg_southeast alaska,reg_stockton,reg_susanville,reg_texarkana,reg_tucson,reg_tuscaloosa,reg_ventura county,reg_visalia-tulare,"reg_washington, DC",reg_western slope,reg_worcester / central MA,reg_yuba-sutter,reg_yuma,fuel_diesel,fuel_electric,fuel_gas,fuel_hybrid,fuel_other,man_acura,man_alfa-romeo,man_aston-martin,man_audi,man_bmw,man_buick,man_cadillac,man_chevrolet,man_chrysler,man_datsun,man_dodge,man_ferrari,man_fiat,man_ford,man_gmc,man_harley-davidson,man_honda,man_hyundai,man_infiniti,man_jaguar,man_jeep,man_kia,man_land rover,man_lexus,man_lincoln,man_mazda,man_mercedes-benz,man_mercury,man_mini,man_mitsubishi,man_nissan,man_pontiac,man_porsche,man_ram,man_rover,man_saturn,man_subaru,man_tesla,man_toyota,man_volkswagen,man_volvo,title_clean,title_lien,title_missing,title_parts only,title_rebuilt,title_salvage,trans_automatic,trans_manual,trans_other,type_SUV,type_bus,type_convertible,type_coupe,type_hatchback,type_mini-van,type_offroad,type_other,type_pickup,type_sedan,type_truck,type_van,type_wagon,state_ak,state_al,state_ar,state_az,state_ca,state_co,state_ct,state_dc,state_de,state_fl,state_ma,state_nc,state_ny,state_or,state_pa,state_tx,state_wa,state_wi
0,6000,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,11900,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

In [37]:
df.isna().sum()[df.isna().sum() > 0]

year         428
odometer    1521
lat          539
long         539
dtype: int64

In [38]:
for column in ['lat', 'long', 'year', 'odometer']:
    df[column] = df[column].fillna(df[column].mean())

In [39]:
df

,price,year,odometer,lat,long,reg_SF bay area,reg_anchorage / mat-su,reg_auburn,reg_bakersfield,reg_bellingham,reg_birmingham,reg_boulder,reg_chico,reg_colorado springs,reg_daytona beach,reg_delaware,reg_denver,reg_dothan,reg_eastern CO,reg_eastern CT,reg_el paso,reg_erie,reg_fairbanks,reg_fayetteville,reg_flagstaff / sedona,reg_florence / muscle shoals,reg_florida keys,reg_fort collins / north CO,reg_fort smith,reg_fresno / madera,reg_ft myers / SW florida,reg_gadsden-anniston,reg_gainesville,reg_gold country,reg_greensboro,reg_hanford-corcoran,reg_hartford,reg_heartland florida,reg_high rockies,reg_hudson valley,reg_humboldt county,reg_huntsville / decatur,reg_imperial county,reg_inland empire,reg_jacksonville,reg_jonesboro,reg_kenai peninsula,reg_la crosse,reg_little rock,reg_los angeles,reg_medford-ashland,reg_mendocino county,reg_merced,reg_mobile,reg_modesto,reg_mohave county,reg_monterey bay,reg_montgomery,reg_new haven,reg_northwest CT,reg_orange county,reg_palm springs,reg_phoenix,reg_prescott,reg_pueblo,reg_redding,reg_reno / tahoe,reg_sacramento,reg_san diego,reg_san luis obispo,reg_santa barbara,reg_santa maria,reg_show low,reg_sierra vista,reg_siskiyou county,reg_skagit / island / SJI,reg_southeast alaska,reg_stockton,reg_susanville,reg_texarkana,reg_tucson,reg_tuscaloosa,reg_ventura county,reg_visalia-tulare,"reg_washington, DC",reg_western slope,reg_worcester / central MA,reg_yuba-sutter,reg_yuma,fuel_diesel,fuel_electric,fuel_gas,fuel_hybrid,fuel_other,man_acura,man_alfa-romeo,man_aston-martin,man_audi,man_bmw,man_buick,man_cadillac,man_chevrolet,man_chrysler,man_datsun,man_dodge,man_ferrari,man_fiat,man_ford,man_gmc,man_harley-davidson,man_honda,man_hyundai,man_infiniti,man_jaguar,man_jeep,man_kia,man_land rover,man_lexus,man_lincoln,man_mazda,man_mercedes-benz,man_mercury,man_mini,man_mitsubishi,man_nissan,man_pontiac,man_porsche,man_ram,man_rover,man_saturn,man_subaru,man_tesla,man_toyota,man_volkswagen,man_volvo,title_clean,title_lien,title_missing,title_parts only,title_rebuilt,title_salvage,trans_automatic,trans_manual,trans_other,type_SUV,type_bus,type_convertible,type_coupe,type_hatchback,type_mini-van,type_offroad,type_other,type_pickup,type_sedan,type_truck,type_van,type_wagon,state_ak,state_al,state_ar,state_az,state_ca,state_co,state_ct,state_dc,state_de,state_fl,state_ma,state_nc,state_ny,state_or,state_pa,state_tx,state_wa,state_wi
0,6000,2010.944261,96449.948933,36.794858,-107.618131,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,11900,2010.944261,96449.948933,36.794858,-107.618131,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fa

#### Splitting and Scaling

In [40]:
y = df.loc[:, 'price']
X = df.drop('price', axis=1)

In [41]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

X

array([[ 0.00000000e+00,  0.00000000e+00, -1.19235678e-15, ...,
        -7.07124460e-03, -1.04886654e-02, -3.16229347e-03],
       [ 0.00000000e+00,  0.00000000e+00, -1.19235678e-15, ...,
        -7.07124460e-03, -1.04886654e-02, -3.16229347e-03],
       [ 0.00000000e+00,  0.00000000e+00, -1.19235678e-15, ...,
        -7.07124460e-03, -1.04886654e-02, -3.16229347e-03],
       ...,
       [ 8.02786577e-01, -3.63243599e-01, -1.10545551e+00, ...,
        -7.07124460e-03, -1.04886654e-02, -3.16229347e-03],
       [ 3.04516573e-01,  1.61174040e-02, -1.10545551e+00, ...,
        -7.07124460e-03, -1.04886654e-02, -3.16229347e-03],
       [-2.18683345e+00,  6.31070584e-02, -1.35637874e+00, ...,
        -7.07124460e-03, -1.04886654e-02, -3.16229347e-03]])

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=34)

In [43]:
X_train.shape, X_test.shape

((70000, 174), (30000, 174))

### Training

In [44]:
lin_model = LinearRegression()

lin_model.fit(X_train, y_train)

lin_y_preds = lin_model.predict(X_test)

In [45]:
lgb_model = lgb.LGBMRegressor(
    boosting_type='gbdt',
    num_leaves = 31,
    n_estimators = 100,
    reg_lambda = 1.0
)

lgb_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038016 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1280
[LightGBM] [Info] Number of data points in the train set: 70000, number of used features: 153
[LightGBM] [Info] Start training from score 33229.276900


LGBMRegressor(reg_lambda=1.0)

In [46]:
lgb_y_preds = lgb_model.predict(X_test)

/home/aditya/miniconda3/envs/dsenv/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [48]:
lin_loss = np.sqrt(mean_squared_error(y_test, lin_y_preds))
lgb_loss = np.sqrt(mean_squared_error(y_test, lgb_y_preds))

In [49]:
print("Linear Regression RMSE: ", lin_loss)
print("Gradient Boosted RMSE: ", lgb_loss)

Linear Regression RMSE:  26253193.522271834
Gradient Boosted RMSE:  26263154.45581862


In [50]:
print("Linear Regression R^2 Score: ", lin_model.score(X_test, y_test))
print("Gradient Boosted R^2 Score: ", lgb_model.score(X_test, y_test))

Linear Regression R^2 Score:  -0.00012880218083433093
Gradient Boosted R^2 Score:  -0.0008878798488758388


/home/aditya/miniconda3/envs/dsenv/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
